In [1]:
import numpy as np
import random
import pandas as pd

In [2]:
def sigmoid(z):
    return 1/(1+np.exp(-z))


In [3]:
def sigmoid_prime(z):
    return  sigmoid(z)*(1 - sigmoid(z))

In [4]:
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [5]:
class QuadraticCost(object):
    @staticmethod
    def fn(a,y):
        #Return Quadratic Cost
        return (0.5)*((np.linalg.norm(a-y))**2)
    @staticmethod
    def delta(z,a,y):
        #Return Final Layer delta
        return (a-y)*sigmoid_prime(z)
    
class CrossEntropyCost(object):
    @staticmethod
    def fn(a,y):
        #Return Cross Entropy Cost
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))
    @staticmethod
    def delta(z,a,y):
        #Return Final Layer delta
        return (a-y)

In [6]:
class Network(object):
    def __init__(self, sizes, cost = CrossEntropyCost):
        self.num_layers = len(sizes)
        self.sizes = sizes 
        self.default_weight_initializer()
        self.cost = cost


    def default_weight_initializer(self):
        #Initialise biases using standard normal distr.
        self.biases = [np.random.randn(y,1) for y in self.sizes[1:]]
        #Initialise weights using standard normal distr. over sqrt of number of input neurons, to ensure network learns faster
        self.weights = [(np.random.normal(0, np.sqrt(x), (y,x))) for x,y in zip(self.sizes[:-1], self.sizes[1:])]

    def large_weight_initializer(self):
        #Initialise biases using standard normal distr.
        self.biases = [np.random.randn(y,1) for y in self.sizes[1:]]
        #Initialise weights using standard normal distr.
        self.weights = [np.random.randn(y,x) for x,y in zip(self.sizes[:-1], self.sizes[1:])]

    
    def feedforward(self, x):
        #Returns output activation corresponding to a given input
        for w,b in zip(self.weights, self.biases):
            x = sigmoid(np.dot(w,x) + b)
        return x
    
    def SGD(self, training_data, epochs, minibatchsize, eta, lmbda = 0.0, evaluation_data = None, monitor_evaluation_cost = False, monitor_evaluation_accuracy = False, monitor_training_cost = False, monitor_training_accuracy = False):
        #Applies stochastic gradient descent on minibatches of the desired size, implimenting L2 regularization. Monitors required quantities if necessary
        #Returns a epoch-wise tuple of evaluation and training cost and accuracy respectively
        if evaluation_data: n_eval = len(evaluation_data)
        n = len(training_data)
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in range(0, epochs):
            #Selecting minibatches randomly
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+minibatchsize] for k in range(0,n,minibatchsize)]

            #Updating weights and biases for each minibatch
            for minibatch in mini_batches:
                self.update_wnb(minibatch, eta, lmbda, len(training_data))
            print("\n", "Epoch {0} completed".format(j))

            if monitor_training_cost:
                #Prints epoch wise cost on training data
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print("Cost on training data is: {}".format(cost))

            if monitor_training_accuracy:
                #Prints epoch wise accuracy on training data
                accuracy = self.accuracy(training_data, convert = True)
                training_accuracy.append(accuracy)
                print("Accuracy on training data is: {}".format(accuracy, n))

            if monitor_evaluation_cost:
                #Prints epoch wise cost on evaluation data
                cost = self.total_cost(evaluation_data, lmbda, convert = True)
                evaluation_cost.append(cost)
                print("Cost on evaluation data is: {}".format(cost))

            if monitor_evaluation_accuracy:
                #Prints epoch wise accuracy on evaluation data
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print("Accuracy on evaluation data is: {}".format(accuracy, n_eval))
        return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy
        


    def update_wnb(self, minibatch, eta, lmbda, n):
        #delta_b and delta_w are layer by layer lists of derivative of cost fn. w.r.t. biases and weights respectively, summed over the minibatch
        delta_b = [np.zeros(b.shape) for b in self.biases]
        delta_w = [np.zeros(w.shape) for w in self.weights]
        for x,y in minibatch:
            #Backpropogation for each input element in minibatch
            D_delta_b, D_delta_w = self.backprop(x,y)
            delta_b = [db +Ddb  for db, Ddb in zip(delta_b, D_delta_b)]
            delta_w = [dw +Ddw for dw, Ddw in zip(delta_w, D_delta_w)]
        #Update weights including L2 regularization
        self.weights = [(1-lmbda*(eta/n))*w - (eta/len(minibatch))*dw for w,dw in zip(self.weights, delta_w)]
        self.biases = [b - (eta/len(minibatch))*db for b,db in zip(self.biases, delta_b)]
    
    def backprop(self, x, y):
        #Returns layer by layer lists of derivative of cost fn. w.r.t. biases and weights, as a tuple
        activation = x
        activations = [x]
        zs = []
        D_delta_b = [np.zeros(b.shape) for b in self.biases]
        D_delta_w = [np.zeros(w.shape) for w in self.weights]
        #Feedforward
        for w,b in zip(self.weights, self.biases):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        #Computing derivative of cost fn. w.r.t. final layer weighted inputs
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        D_delta_b[-1] = delta
        D_delta_w[-1] = np.dot(delta, activations[-2].transpose())
        #Backpropogating the error
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            D_delta_b[-l] = delta
            D_delta_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (D_delta_b, D_delta_w)

    

    def total_cost(self, data, lmbda, convert = False):
        #Returns total cost on data including L2 regularization
        #Convert is false for training data, and true for evaluation/test data. Evaluation and test data use integers as expected outputs, training data uses vector as output
        cost = 0.0

        for x,y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a,y)/len(data)
        
        #Includes regularised cost
        cost += (0.5)*(lmbda/len(data))*sum([np.linalg.norm(w)**2 for w in self.weights])
        return cost


    def accuracy(self, data, convert = False):
        #Returns number of inputs for which the neural network gives the correct output
        #Convert is false for evaluation/test data, and true for training data
        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y))   for (x,y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y)   for (x,y) in data]
        return sum([int(x==y) for (x,y) in results])/len(data)